# SPDE simulation on a sphere

<!-- SUMMARY: Simulations performed on a Sphere, treated in the SPDE formalism -->

<!-- CATEGORY: SPDE -->

In [ ]:
import gstlearn as gl
import gstlearn.plot as gp
import gstlearn.document as gdoc
import gstlearn.plot3D as gop
import numpy as np 
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import IPython
from IPython.display import Markdown
gdoc.setNoScroll()

In [ ]:
Markdown(gdoc.loadDoc("SPDE_Sphere.md"))

In [ ]:
# Markov (if False, Matérn covariance will be used)
Markov = True

### Parametrization

In [ ]:
#Sphere radius
R = gl.EARTH_RADIUS
gl.defineDefaultSpace(gl.ESpaceType.SN,param=R)

#Scale parameter (for convenience, it is defined as the proportion of the radius)
ratioRange = 0.2
scale = R * ratioRange

# sill 
sill = 2. 

# Smoothness parameter (for Matérn case)
nu = 2

# Markov coefficients (for Markov case)
coeffs = [1,-1,.5]

### Meshing

In [ ]:
mesh = gl.MeshSphericalExt()
err = mesh.resetFromDb(None,None,triswitch = "-r5",verbose=False)

Sampling Db creation

In [ ]:
nsample = 4000
#sub-sampling to reduce computational burden
np.random.seed(123)
ind = np.random.choice(mesh.getNApices(),size=nsample,replace=False)

#Creation of the db
X = mesh.getCoordinatesPerApex(0)
Y = mesh.getCoordinatesPerApex(1)

dbdat = gl.Db.create()
dbdat["x"] = np.array(X)[ind]
dbdat["y"] = np.array(Y)[ind]
dbdat.setLocators(["x","y"],gl.ELoc.X)

varsize = np.ones(nsample)
iuid = dbdat.addColumns(varsize, "sizes")
varcolor = np.ones(nsample)
iuid = dbdat.addColumns(varcolor, "colors")

### Covariance model

The covariance model is Markov or Matérn.

In [ ]:
if Markov : 
    model = gl.Model.createFromParam(type=gl.ECov.MARKOV,
                                 range = scale,
                                 sill = sill,
                                 flagRange= False)
    model.setMarkovCoeffs(0, coeffs)
    
else :

    model = gl.Model.createFromParam(type=gl.ECov.MATERN,
                                 range = scale,
                                 sill = sill,
                                 param=nu,
                                 flagRange= False)

### Precision matrix

In [ ]:
Q = gl.PrecisionOp(mesh,model.getCovAniso(0))

### Simulation 

In [ ]:
result = np.array(Q.simulateOne())

### Display the realization
 

In [ ]:
surface = gop.SurfaceOnMesh(mesh, result,opacity=1)
fig = go.Figure(data=[ surface ])
fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False )
f = fig.show()

### Compute covariance (of discretized solution)

We use the fact that $\Sigma = Q^{-1}$ and solve $Qx = e_j$ for an arbitrary index $j$.

**Get the distances**

In [ ]:
ind0 = 12
distances = np.array(mesh.getDistances(ind0))

**Compute the covariances**

In [ ]:
covDiscr = np.array(Q.computeCov(ind0))

**Sort for the plot**

In [ ]:
covDiscrClose = covDiscr[np.argsort(distances)]
deltaLong =  np.sort(distances)
print(f"Discretized Covariance = {round(covDiscrClose[0],4)}")

**Display the result**

In [ ]:
plt.plot(deltaLong,covDiscrClose,"--",label = "Discretized covariance")
ax = plt.legend()
print(f"Discretized variance = {round(covDiscrClose[0],4)}")

### Variogram of the realization

The empirical variogram is computed by using the great-circle distance.

In [ ]:
nlag = 50 # number of discretization points
dlag = 0.04 # lag with respect to the unit sphere (it will be multiplied
# by R in the creation of the VarioParam.

dbdat["simu"] = np.array(result)[ind]
dbdat.setLocators(["simu"],gl.ELoc.Z)

#Variogram 

vp = gl.VarioParam.createOmniDirection(nlag=nlag,dlag=dlag * R)
vario = gl.Vario.create(vp)
ax = vario.compute(dbdat,gl.ECalcVario.VARIOGRAM)
#vario.display()

In [ ]:
res = gp.plot(vario, label = "Empirical Variogram", flagLegend=True)

### Theoretical covariance function

In [ ]:
Markdown(gdoc.loadDoc("Covariance_Sphere.md"))

### Evaluation 

In [ ]:
ndisc = 100 # number of discretization steps for the covariance 
N = 20 # size of the decomposition

h = np.linspace(0,np.max(deltaLong),ndisc)
a = model.getCovAniso(0)
uu = np.array([a.evalCovOnSphere(i,N) for i in h]) # modif dR

gp.plot(vario, label = "Empirical Variogram", flagLegend=True)
plt.plot(h, sill - uu,label = "Theoretical Variogram")
plt.plot(deltaLong,covDiscrClose[0] - covDiscrClose,"--",label = "Discretized model")
plt.show()

There is a slight difference between the theoretical variogram and the one obtained from the SPDE discretization due to a numerical error on the variance introduced by the discretization. The comparison of the covariance shows that this numerical error is rather small :

In [ ]:
h = np.linspace(0,np.max(deltaLong),ndisc)
vario = gl.Vario.create(vp)

ax = vario.compute(dbdat,gl.ECalcVario.COVARIANCE)
#ax = gp.variogram(vario,label = "Empirical Covariance")
ax = plt.plot(h, uu, label = "Theoretical Covariance")
plt.plot(deltaLong,covDiscrClose,"--",label = "Discretized model")
ax = plt.legend()
plt.show()

print(f"Theoretical variance = {round(uu[0],4)}")

## Kriging

Plotting the mesh and the data

In [ ]:
dbdat

In [ ]:
mesh = gl.MeshSphericalExt()
err = mesh.resetFromDb(None,None,triswitch = "-r2",verbose=False)

point = gop.PointDb(dbdat, size=1, nameColor = "simu", fromLongLat=True)
blank = gop.SurfaceOnMesh(mesh, opacity=1)
meshing = gop.Meshing(mesh)

fig = go.Figure(data = [blank, meshing, point])
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False )
f = fig.show()

In [ ]:
ball = gl.Ball(mesh);
ball.display();

Highlight One sample and check the closest meshes

In [ ]:
point = gop.PointDb(dbdat, size=1, nameColor = "simu", fromLongLat=True)
blank = gop.SurfaceOnMesh(mesh, opacity=1)
meshing = gop.Meshing(mesh)

fig = go.Figure(data = [blank, meshing, point])
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False )
f = fig.show()

In [ ]:
def highlight(sample = 0, verbose = False):
    if verbose:
        print("Rank of the Target sample =", sample)
        
    target = dbdat.getSampleCoordinates(sample)
    if verbose:
        print("Target = ",target)
        
    mymesh = ball.queryClosest(target)
    if verbose:
        print("Rank of the Target Mesh = ", mymesh)

    veclon = mesh.getCoordinatesPerMesh(mymesh, 0)
    veclat = mesh.getCoordinatesPerMesh(mymesh, 1)
    if verbose:
        print("Longitude = ", veclon)
        print("Latitude  = ", veclat)
    
    return veclon, veclat

In [ ]:
mysample = 5
veclon, veclat = highlight(mysample)
dbdat.setValue("sizes", mysample, 12)
dbdat.setValue("colors", mysample, 1)
point   = gop.PointDb(dbdat, nameSize = "sizes", nameColor = "colors", fromLongLat=True)
blank   = gop.SurfaceOnMesh(mesh, opacity=1,)
meshing = gop.Meshing(mesh)
scatter = gop.ScatterOnSphere(veclon, veclat, mode="markers", m_color='black', m_size=2)

fig = go.Figure(data = [blank, point, meshing, scatter])
fig.update_scenes(xaxis_visible=False, yaxis_visible=False, zaxis_visible=False )
f = fig.show()

dbdat.setValue("sizes", mysample, 1)
dbdat.setValue("colors", mysample, 1)

In [ ]:
Aproj = gl.ProjMatrix(dbdat, mesh)
Aproj.dumpVerticesUsed(30) # Dump the weights for the first samples